In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import scipy.stats as st
import os.path
from os import path
import json

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_players = "output_data/players.csv"

# Range of latitudes and longitudes
# lat_range = (-90, 90)
# lng_range = (-180, 180)

In [2]:
def get_titledplayers_api(playertypes):
    url = "https://api.chess.com/pub/titled/"
    
    i = 0        
    players_data = []
    
    for playertype in playertypes:
        print(f"Index {str(i)} fetching player list for titled player type {playertype}")
    
        # Build query URL
        query_url = f"{url}{playertype}"

        # increment index counter
        i += 1

        # Get weather data    
        playertype_response = requests.get(query_url)
        playertype_json = playertype_response.json()
        players_data.append(playertype_json)        
        time.sleep(1)    
        
    return players_data

In [3]:
def get_playerdata_api(player_usernames):
    url = "https://api.chess.com/pub/player/"
    
    i = 0        
    player_data = []
    
    for player_username in player_usernames:
        print(f"Index {str(i)} fetching player data for player {player_username}")
    
        # Build query URL
        query_url = f"{url}{player_username}"

        # increment index counter
        i += 1

        # Get player data    
        try:
            player_response = requests.get(query_url)
            player_json = player_response.json()
            player_data.append(player_json)        
            time.sleep(1)    
        except:
            print("unexpected error")
        
        # if i > 10:
        #    break
    return player_data

In [4]:
titled_players_type = ["GM","WGM","IM","WIM","FM","WFM","NM","WNM","CM","WCM"]

In [5]:
playerslist_data = get_titledplayers_api(titled_players_type)

Index 0 fetching player list for titled player type GM
Index 1 fetching player list for titled player type WGM
Index 2 fetching player list for titled player type IM
Index 3 fetching player list for titled player type WIM
Index 4 fetching player list for titled player type FM
Index 5 fetching player list for titled player type WFM
Index 6 fetching player list for titled player type NM
Index 7 fetching player list for titled player type WNM
Index 8 fetching player list for titled player type CM
Index 9 fetching player list for titled player type WCM


In [6]:
playerlist_index = 0
i = 0

player_usernames = []

for playerlist in playerslist_data:    
    for player in playerlist["players"]:        
        player_usernames.append(player)
        
    playerlist_index += 1
    

```{
  "avatar": "https://images.chesscomfiles.com/uploads/v1/user/14775486.9072453d.200x200o.50bb2d10a7d5.jpeg",
  "player_id": 14775486,
  "@id": "https://api.chess.com/pub/player/140mariam",
  "url": "https://www.chess.com/member/140mariam",
  "name": "Mariam Avetisyan",
  "username": "140mariam",
  "title": "WFM",
  "followers": 56,
  "country": "https://api.chess.com/pub/country/AM",
  "last_online": 1609780423,
  "joined": 1385829847,
  "status": "premium",
  "is_streamer": false
}
```

In [7]:
len(player_usernames)

9302

In [8]:
# check if output_data_file exists already, if it doesn't exist, do api call to get weather data
if (path.exists(output_data_players)):
    # ****************
    # pull prevously generated and saved output data from csv
    chessplayer_df = pd.read_csv(output_data_players)
else:    
    # run api calls for each chess.com username
    # this step is skipped if the output_data_file already exists
    # !!!!! DELETE CSV OUTPUT FILE to generate new playerdata    
    playerdata = get_playerdata_api(player_usernames)
    
    player_id = []
    username = []
    title = []
    followers = []
    country = []
    location = []
    last_online = []
    joined = []
    status = []
    is_streamer = []

    i = 0

    for player in playerdata: 
        try:
            un = player["username"]
            print(f"Index {i} processing player {un}")
            player_id.append(player["player_id"])
            username.append(player["username"])
            title.append(player["title"])
            followers.append(player["followers"])
            country.append(player["country"].replace("https://api.chess.com/pub/country/","")),
            if "location" in player.keys():
                location.append(player["location"])        
            else:
                location.append("")                
            last_online.append(player["last_online"])
            joined.append(player["joined"])
            status.append(player["status"])
            is_streamer.append(player["is_streamer"])
        except:
            print("unexpected error")

        i += 1 
        
    player_dict = {
        "player_id":player_id,
        "username":username,
        "title":title,
        "followers":followers,
        "country":country,
        "location":location,
        "last_online":last_online,
        "joined":joined,
        "status":status,
        "is_streamer":is_streamer
    }
    chessplayer_df = pd.DataFrame(player_dict)

    chessplayer_df.to_csv(output_data_players, index=True,index_label="player_index")  
    
    

In [9]:
chessplayer_df

,player_index,player_id,username,title,followers,country,location,last_online,joined,status,is_streamer
0,0,18800602,123lt,GM,32,CN,NaN,1608465021,1410059361,premium,False
1,1,29499974,124chess,GM,79,RU,Красноярск,1609836300,1471316272,premium,False
2,2,30610578,1977ivan,GM,95,RS,Belgrade,1609872690,1477565847,premium,False
3,3,13013662,1stsecond,GM,734,LV,Riga,1609536323,1375617889,premium,False
4,4,39230304,2nd_life,GM,22,CH,Zurich,1609876221,1507311109,premium,False
...,...,...,...,...,...,...,...,...,...,...,...
9295,9295,29980036,yashami,WCM,21,NP,Kathmandu,1594232302,1474135436,premium,False
9296,9296,67624176,yoriiallam,WCM,10,EG,NaN,1609501122,1573057158,premium,False
9297,9297,86574494,zaineb218,WCM,3,TN,Tunisie,1609446669,1594630390,premium,False
9298,9298,85888234,zarinur,WCM,15,KZ,NaN,1609904817,1594013985,premium,False
